<a href="https://colab.research.google.com/github/francisco140472/Tamanduatei_Vu_Enorsul.github.io/blob/main/JOTFORM_VU_TAMANDUATI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install schedule
!pip install paramiko
!git clone https://github.com/francisco140472/Tamanduatei_Vu_Enorsul.github.io

Cloning into 'Tamanduatei_Vu_Enorsul.github.io'...
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 11 (delta 2), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (11/11), 97.03 KiB | 8.82 MiB/s, done.
Resolving deltas: 100% (2/2), done.


In [9]:
%cd Tamanduatei_Vu_Enorsul.github.io


/content/Tamanduatei_Vu_Enorsul.github.io


In [10]:
!cp JOTFORM VU - TAMANDUATI.ipynb .


cp: cannot stat 'JOTFORM': No such file or directory
cp: cannot stat 'VU': No such file or directory
cp: cannot stat '-': No such file or directory
cp: cannot stat 'TAMANDUATI.ipynb': No such file or directory


In [11]:
!git add .

In [12]:
!git commit -m "Adicionando o código inicial"


Author identity unknown

*** Please tell me who you are.

Run

  git config --global user.email "you@example.com"
  git config --global user.name "Your Name"

to set your account's default identity.
Omit --global to set the identity only in this repository.

fatal: unable to auto-detect email address (got 'root@74c6413dd4ac.(none)')


In [14]:
!git push origin main


fatal: could not read Username for 'https://github.com': No such device or address


In [15]:
!git config --global user.name "francisco140472"
!git config --global user.email "franciscosales.1404@gmail.com"


In [17]:
import requests
import pandas as pd
import folium
from io import BytesIO
import paramiko
import time
from datetime import datetime

# URL para o primeiro arquivo JotForm
url_ocorrencias = "https://www.jotform.com/excel/242924996615066"

# Fazendo o download do primeiro arquivo Excel
response_ocorrencias = requests.get(url_ocorrencias)

# Verificando se o download foi bem-sucedido
if response_ocorrencias.status_code == 200:
    # Lê o conteúdo do Excel diretamente no pandas
    df = pd.read_excel(BytesIO(response_ocorrencias.content))
else:
    print(f"Error ao baixar ocorrências: {response_ocorrencias.status_code}")

# Remove registros onde NÚCLEO_DE_ATUAÇÃO é NaN
df = df.dropna(subset=['NÚCLEO_DE_ATUAÇÃO'])

# Contagem de registros não nulos em cada coluna
total_ocorrencias = df['OCORRÊNCIA DE CAMPO CADASTRO'].notna().sum()
soma_caixa_uma = df['OCORRÊNCIA DE CAMPO CAIXA UMA'].notna().sum()
total_quantidade_instalacao = df['OCORRÊNCIA DE CAMPO LIGAÇÃO'].notna().sum()

# URL para o segundo arquivo JotForm com dados de redes
url_area_atuacao = "https://www.jotform.com/excel/243094706627057"

# Fazendo o download do segundo arquivo Excel
response_area = requests.get(url_area_atuacao)

# Verificando se o download foi bem-sucedido
if response_area.status_code == 200:
    # Lê o conteúdo do Excel diretamente no pandas
    df_area = pd.read_excel(BytesIO(response_area.content))
else:
    print(f"Error ao baixar área de atuação: {response_area.status_code}")

# Calcula o total de rede de água e esgoto se as colunas existirem

###################
# Certifique-se de que as colunas de rede estão em formato numérico
df_area['TOTAL DE REDE ESGOTO 200'] = pd.to_numeric(df_area['TOTAL DE REDE ESGOTO 200'], errors='coerce')
df_area['TOTAL DE REDE 110 ÁGUA'] = pd.to_numeric(df_area['TOTAL DE REDE 110 ÁGUA'], errors='coerce')

# Soma de redes
total_rede_esgoto = df_area['TOTAL DE REDE ESGOTO 200'].sum()
total_rede_agua = df_area['TOTAL DE REDE 110 ÁGUA'].sum()

# Verifique se as colunas 'NÚCLEO_DE_ATUAÇÃO' e 'AREA_DE_ATUACAO' estão presentes em cada DataFrame
# e renomeie 'AREA_DE_ATUACAO' se necessário para que o merge ocorra sem erros.
##########################################################

##########################################################
## Renomeie 'AREA_DE_ATUACAO' se necessário
if 'AREA_DE_ATUACAO' in df_area.columns:
    df_area = df_area.rename(columns={'AREA_DE_ATUACAO': 'NÚCLEO_DE_ATUAÇÃO'})

# Verifique se a coluna 'NÚCLEO_DE_ATUAÇÃO' existe em ambos os DataFrames
if 'NÚCLEO_DE_ATUAÇÃO' in df.columns and 'NÚCLEO_DE_ATUAÇÃO' in df_area.columns:
    # Garanta que os tipos de dados sejam compatíveis
    df['NÚCLEO_DE_ATUAÇÃO'] = df['NÚCLEO_DE_ATUAÇÃO'].astype(str)
    df_area['NÚCLEO_DE_ATUAÇÃO'] = df_area['NÚCLEO_DE_ATUAÇÃO'].astype(str)

    # Execute o merge
    df = df.merge(df_area[['NÚCLEO_DE_ATUAÇÃO', 'REDE ESGOTO 200', 'REDE 110 ÁGUA']],
                  on='NÚCLEO_DE_ATUAÇÃO', how='left')
else:
    print("A coluna 'NÚCLEO_DE_ATUAÇÃO' não existe em um dos DataFrames.")


# Agrupamento por NÚCLEO_DE_ATUAÇÃO para marcadores
localizacoes_por_bairro = {}
for bairro in df['NÚCLEO_DE_ATUAÇÃO'].unique():
    dados_bairro = df[df['NÚCLEO_DE_ATUAÇÃO'] == bairro]
    localizacoes = []
    for index, row in dados_bairro.iterrows():
        if pd.notna(row['Geolocation']) and ',' in row['Geolocation']:
            try:
                lat, lng = map(float, row['Geolocation'].split(','))
                foto_url = row.get('FOTO_DO_IMOVEL', 'https://via.placeholder.com/150')
                localizacoes.append((lat, lng, row['BAIRRO'],
                                     row.get('FUNCIONÁRIO CADASTRO', 'Não informado'),
                                     row.get('OCORRÊNCIA DE CAMPO CADASTRO', 'Não informada'),
                                     row.get('IDENTIFICAÇÃO ÚNICA', 'Não informada'),
                                     foto_url))
            except ValueError:
                print(f"Erro ao converter coordenadas: {row['Geolocation']}")
    localizacoes_por_bairro[bairro] = localizacoes

# Configurações de cores e ícones para bairros
cores_icones = ['red', 'blue', 'green', 'purple', 'orange', 'darkred', 'lightred', 'beige', 'darkblue', 'darkgreen']
icons = ['info-sign', 'cloud', 'leaf', 'star', 'ok', 'bookmark', 'cutlery', 'coffee', 'road', 'home']
cores_bairro = {bairro: (cores_icones[i % len(cores_icones)], icons[i % len(icons)]) for i, bairro in enumerate(localizacoes_por_bairro)}

# Cria o mapa
m = folium.Map(location=[-23.55, -46.63], zoom_start=12)

# Adiciona título ao mapa
title_html = '''
    <h3 align="center" style="font-size:20px"><b>VU - TAMANDUATEI - ENORSIG</b></h3>
'''
m.get_root().html.add_child(folium.Element(title_html))

# Tabela de resumo de ocorrências e redes
tabela_resumo_html = '''
<div style="position: fixed;
            top: 80px; left: 50px; width: 300px;
            background-color: white;
            padding: 10px;
            border: 2px solid grey;
            z-index: 9999;">
    <h4>Resumo de Ocorrências e Redes</h4>
    <table style="width: 100%; border-collapse: collapse; text-align: center;">
        <tr>
            <th>Ocorrência</th>
            <th>Caixa Uma</th>
            <th>Ligações</th>
            <th>Rede Água</th>
            <th>Rede Esgoto</th>
        </tr>
        <tr>
            <td>{total_ocorrencias}</td>
            <td>{soma_caixa_uma}</td>
            <td>{total_quantidade_instalacao}</td>
            <td>{total_rede_agua}</td>
            <td>{total_rede_esgoto}</td>
        </tr>
    </table>
</div>
'''
m.get_root().html.add_child(folium.Element(tabela_resumo_html.format(
    total_ocorrencias=total_ocorrencias,
    soma_caixa_uma=soma_caixa_uma,
    total_quantidade_instalacao=total_quantidade_instalacao,
    total_rede_agua=total_rede_agua,
    total_rede_esgoto=total_rede_esgoto
)))
########################################################
# Adicionar contagem por tipo de "OCORRÊNCIA DE CAMPO CADASTRO"
total_por_tipo = df['OCORRÊNCIA DE CAMPO CADASTRO'].value_counts()

# Adicionar contagem de cadastro por dia, mantendo apenas a última data
df['DATA CADASTRO'] = pd.to_datetime(df['DATA CADASTRO'], errors='coerce')
total_por_dia = df.groupby(df['DATA CADASTRO'].dt.date).size()
ultima_data = total_por_dia.index[-1] if not total_por_dia.empty else None
total_ultima_data = total_por_dia[ultima_data] if ultima_data else 0

# Tabela HTML com total por tipo e apenas a última data, abaixo da primeira tabela
tabela_tipo_ultima_data_html = '''
<div style="position: fixed;
            top: 320px; left: 50px; width: 300px;
            background-color: white;
            padding: 10px;
            border: 2px solid grey;
            z-index: 9999;">
    <h4>Resumo por Tipo</h4>
    <table style="width: 100%; border-collapse: collapse; text-align: center;">
        <tr>
            <th style="border: 1px solid black; padding: 5px; text-align: center;">Tipo de Ocorrência</th>
            <th style="border: 1px solid black; padding: 5px; text-align: center;">Total</th>
        </tr>''' + ''.join([f'''
        <tr>
            <td style="border: 1px solid black; padding: 5px;">{tipo}</td>
            <td style="border: 1px solid black; padding: 5px;">{total}</td>
        </tr>''' for tipo, total in total_por_tipo.items()]) + '''
    </table>




    <h4>Última Data</h4>
    <table style="width: 100%; border-collapse: collapse; text-align: center;">
        <tr>
            <th style="border: 1px solid black; padding: 5px; text-align: center;">Data</th>
            <th style="border: 1px solid black; padding: 5px; text-align: center;">Total</th>
        </tr>
        <tr>
            <td style="border: 1px solid black; padding: 5px;">{ultima_data}</td>
            <td style="border: 1px solid black; padding: 5px;">{total_ultima_data}</td>
        </tr>
    </table>

</div>
'''

# Adiciona a tabela de tipo e última data ao mapa
m.get_root().html.add_child(folium.Element(tabela_tipo_ultima_data_html.format(
    ultima_data=ultima_data.strftime('%Y-%m-%d') if ultima_data else 'N/A',
    total_ultima_data=total_ultima_data
)))


#################################3

# Adicionar marcadores para cada núcleo e camada
for bairro, localizacoes in localizacoes_por_bairro.items():
    cor, icone = cores_bairro[bairro]
    feature_group = folium.FeatureGroup(name=bairro)
    for lat, lng, bairro, funcionario, cadastro, identificacao, foto_url in localizacoes:
        popup_html = f"""
            <div>
                <h4>{bairro}</h4>
                <p><b>Funcionário:</b> {funcionario}</p>
                <p><b>Cadastro:</b> {cadastro}</p>
                <p><b>Identificação Única:</b> {identificacao}</p>
                <img src="{foto_url}" alt="Foto do Imóvel" width="100%">
            </div>
        """
        folium.Marker(
            location=[lat, lng],
            popup=folium.Popup(popup_html, max_width=300),
            icon=folium.Icon(color=cor, icon=icone)
        ).add_to(feature_group)
    feature_group.add_to(m)

# Controle de camadas
folium.LayerControl().add_to(m)

# Salva o mapa em HTML
m.save("index.html")
print("HTML gerado com sucesso!")

#########################################################################################


A coluna 'NÚCLEO_DE_ATUAÇÃO' não existe em um dos DataFrames.
HTML gerado com sucesso!
